The purpose of this jupyter notebook is to provide some basic building blocks for analyzing these datasets. At this stage, it is still under review and should be read as in its preliminary stages. Please reach out with any comments, edits, suggestions, or questions.

--------

First we will import the necessary packages. To ease our analysis we rely heavily on two popular Python natural language processing frameworks, `nltk` and `spacy`. As part of this import step, we need to download the necessary tokenization and language models packages.

In [1]:
from bs4 import BeautifulSoup
import csv
import logging
import pandas as pd
import requests
import time
import nltk
import spacy
import numpy as np
from nltk.text import Text

####
# import warnings
# warnings.filterwarnings('ignore')

# display max column width for descriptions
pd.set_option('display.max_colwidth', None)

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

!python3 -m spacy download en_core_web_sm

# load small language model
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielevans/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/danielevans/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.1 MB/s eta 0:00:00m eta 0:00:010:0101
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Next we'll read in the raw data. To make sure everything is working correctly, we'll print out the first few lines

In [3]:
df = pd.read_csv("raw.csv")
df.head()

created_published  date  \
0               ['Abbeville, S.C. : Charles H. Allen, 1847-1869.']  1847   
1                                ['Abbeville, S.C. : Hugh Wilson']  1865   
2                          ['Abbeville, S.C. : Bonham and Perrin']  1884   
3         ['Abbeville, S.C. : W.A. Lee & Hugh Wilson, 1869-1924.']  1869   
4  ['Abbeville, S.C. : W.A. Lee and Hugh Wilson, Jr., 1860-1869.']  1860   

  dates_of_publication  \
0            1847-1869   
1            1865-1865   
2            1884-1887   
3            1869-1924   
4            1860-1869   

                                                                                                                                                                                                                                                                                                                                                                                                                                                description  \
0                                                                                                              ['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, no. 47 (Sept. 29, 1869). Publication suspended, <1861-1865>. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Abbeville press (DLC)sn 85042527 (OCoLC)12622573 Abbeville press and banner (DLC)sn 84026853 (OCoLC)10708843']   
1                                                                                                         ['Weekly Began in July 1865. Ceased with Aug. 31, 1865. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Published during the suspension of the Abbeville press. Description based on: Vol. 1, no. 3 (Aug. 10, 1865). Latest issue consulted: Vol. 1, no. 6 (Aug. 31, 1865).']   
2                                                                                                                                  ['Weekly Began Oct. 1, 1884; ceased in 1887. Cf. Watchman and southron, July 20, 1887. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Description based on: Vol. 1, no. 1 (Oct. 1, 1884). Latest issue consulted: Vol. 3, no. 38 (June 28, 1887).']   
3  ['Triweekly, Jan. 7, 1920-Feb. 13, 1924 Began with Oct. 1, 1869; ceased with Feb. 13, 1924. Title varies slightly. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Formed by the union of: Abbeville press and: Abbeville banner. Description based on: Vol. 17, no. 23 (Oct. 1, 1869). Latest issue consulted: Feb. 13, 1924. Press and banner (DLC)  2013228796 (OCoLC)841312126']   
4                                                                                  ['Weekly Vol. 8, no. 28 (Nov. 9, 1860)-v. 17, no. 22 (Sept. 24, 1869). Suspended <1864>-Aug. 1865. Resumed with Sept. 7, 1865 issue. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Abbeville banner (DLC)sn 85026945 (OCoLC)12795764 Abbeville press and banner (DLC)sn 84026853 (OCoLC)10708843']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

We next create a small Python function that removes the html tags from the `essay` column:

In [4]:
def stripped_strings(text):
    try:
        return (' '.join(BeautifulSoup(text).stripped_strings))
    except TypeError as e:
        print(e)


df['essay'] = [stripped_strings(text) for text in df['essay'] ]
    

object of type 'float' has no len()


In [5]:
df.head()

,created_published,date,dates_of_publication,description,essay,essay_contributor,language,latlong,location,raw_lccn,subjects,title,url
0,"['Abbeville, S.C. : Charles H. Allen, 1847-1869.']",1847,1847-1869,"['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, no. 47 (Sept. 29, 1869). Publication suspended, <1861-1865>. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Abbeville press (DLC)sn 85042527 (OCoLC)12622573 Abbeville press and banner (DLC)sn 84026853 (OCoLC)10708843']","For nearly a century, the Abbeville Press and Banner (1844-1925) reported on international, national, and local news for the citizens of Abbeville County, located in the upstate region of South Carolina. The N.W. Ayer & Son’s American Newspaper Annual described Abbeville County simply as a “cotton growing district,” which while true on the surface is also slightly misleading. Although the cultivation of cotton had indeed dominated the county’s economy for most of the 19 th century, in the 1890s Abbeville was undergoing a period of economic growth, starting with the arrival of the Georgia, Carolina, and Northern Railroad (later called the Seaboard Air Line) in 1892 and the founding of the Abbeville Cotton Mill Company in 1895. Other significant developments in the region included: the establishment of an African American school, called Ferguson Academy (later renamed Harbison Agricultural College), in 1868; the founding of Calhoun Falls at the junction of the Seaboard Air Line and Savannah Valley Railroad in the early 1890s; and the creation of Greenwood and McCormick Counties from portions of Abbeville County in 1899 and 1916. The Abbeville Press and Banner began as two weekly newspapers, the Abbeville Banner and Abbeville Press . Charles Henry Allen and Henry S. Kerr had established the Abbeville Banner in 1844. Its competitor, the Abbeville Independent Press (later known as the Abbeville Press ) began in 1853. Coleman C. Puckett and George W. Fant brought considerable experience to their jobs as publishers of the Abbeville Independent Press , Puckett having previously edited the Anderson Southern Rights Advocate and Fant having apprenticed with the Anderson Gazette . During the Civil War, the Abbeville Banner suspended publication. Hugh Wilson, Jr. and William Augustus Lee, the proprietors of the Abbeville Press , turned over publishing duties to a former business partner and enlisted in the Confederate States Army. Sadly, few issues of either newspaper have survived from the 1860s. After the war, Wilson and Lee bought the Abbeville Banner and merged it with the Abbeville Press to create the Abbeville Press and Banner . The first issue appeared on October 1, 1869. Like other white Democrat newspapers of the era, the Press and Banner steered a conservative course, celebrating the return of the “Bourbons,” or antebellum-era aristocrats, to political power in 1877 and championing the interests of agrarian elites. Throughout its existence, it faced only one true competitor, the Abbeville Medium , which existed from 1871 through 1923. Circulation rates for the Press and Banner generally hovered between 800 and 1,200 subscriptions. Several of the men who ran these newspapers were leaders in their communities. George W. Fant, who was briefly involved with the Independent Press , served as postmaster and director of the National Bank of Anderson. Hugh Wilson, Jr., in addition to editing and publishing the Press and Banner , served as director of and stockholder in the Abbeville Cotton Mill and was also an officer in the South Carolina Press Association. Fellow proprietor William Augustus Lee practiced real estate law, represented the Abbeville District in the South Carolina Constitutional Convention of 1865, and served as a state representative in 1865-66. Starting in the 1920s, the Abbeville Press and Banner underwent a series of changes, from adopting a triweekly publication schedule to shortening it

-------

Now that the data has been cleaned, we can begin pre-processing it. Let's [tokenize](https://www.nltk.org/_modules/nltk/tokenize/punkt.html) the essays into their individual components. First, we convert the `essay` column into a string. We'll next break the essays down into individual unigrams and then break those down into tokenized words. Read more about `unigrams` and `word tokenization` here: *GET LINK*

In [4]:
df["essay"] = df["essay"].astype(str)

In [5]:
start = time.time()
df["unigrams"] = df["essay"].apply(nltk.word_tokenize)
print(time.time() - start)

4.43222188949585


Let's print out our unigrams to get a sense of what our code did.

In [6]:
text = df['unigrams'].apply(Text)
text.head()

0                                                             (<, p, >, For, nearly, a, century, ,, the, <, em, >, Abbeville, Press, and, Banner, <, /em, >, (, 1844-1925, ), reported, on, international, ,, national, ,, and, local, news, for, the, citizens, of, Abbeville, County, ,, located, in, the, upstate, region, of, South, Carolina, ., The, <, em, >, N.W, ., Ayer, &, amp, ;, Son, &, #, 8217, ;, s, American, Newspaper, Annual, <, /em, >, described, Abbeville, County, simply, as, a, &, #, 8220, ;, cotton, growing, district, ,, &, #, 8221, ;, which, while, true, on, the, surface, is, also, slightly, misleading, ., Although, the, ...)
1    (<, p, >, The, short-lived, weekly, <, em, >, Abbeville, Bulletin, <, /em, >, (, July-August, 1865, ), offers, a, glimpse, into, a, community, adapting, to, new, realities, under, challenging, circumstances, ., In, the, early, 1860s, ,, the, white, citizens, of, Abbeville, County, (, then, Abbeville, District, ), ,, South, Carolina, ,, had, enthusias

Next we'll tokenize the sentences and apply part of speech tagging.

In [7]:
token_sent = df.essay.apply(nltk.word_tokenize)  
tagged_sent= token_sent.apply(nltk.pos_tag)
tagged_sent

0                                                                        [(<, JJ), (p, NN), (>, NN), (For, IN), (nearly, RB), (a, DT), (century, NN), (,, ,), (the, DT), (<, NNP), (em, NN), (>, NNP), (Abbeville, NNP), (Press, NNP), (and, CC), (Banner, NNP), (<, NNP), (/em, NNP), (>, NNP), ((, (), (1844-1925, JJ), (), )), (reported, VBD), (on, IN), (international, JJ), (,, ,), (national, JJ), (,, ,), (and, CC), (local, JJ), (news, NN), (for, IN), (the, DT), (citizens, NNS), (of, IN), (Abbeville, NNP), (County, NNP), (,, ,), (located, VBN), (in, IN), (the, DT), (upstate, JJ), (region, NN), (of, IN), (South, NNP), (Carolina, NNP), (., .), (The, DT), (<, JJ), (em, NN), (>, NNP), (N.W, NNP), (., .), (Ayer, NNP), (&, CC), (amp, NN), (;, :), (Son, NNP), (&, CC), (#, #), (8217, CD), (;, :), (s, CC), (American, JJ), (Newspaper, NNP), (Annual, NNP), (<, NNP), (/em, NNP), (>, NNP), (described, VBD), (Abbeville, NNP), (County, NNP), (simply, RB), (as, IN), (a, DT), (&, CC), (#, #), (8220, CD), (;, 

Now that we have pre-processed our `essays`, we can do basic things like get the count of words. e.g. 'century'

In [8]:
t = token_sent.apply(lambda x: nltk.Text(x).count('century'))
t

0       2
1       0
2       1
3       2
4       2
       ..
3675    0
3676    0
3677    0
3678    0
3679    0
Name: essay, Length: 3680, dtype: int64

More interesting, perhaps, is the fact that we can create a concordance of words and perform a search. In this example, we concordance search, grab 35 characters before / after word a particular word. Rows with `no matches` denote that the essay does not contain the text in question.

In [9]:
t = token_sent.apply(lambda x: nltk.Text(x).concordance("printer"))
t

no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
 was elected to the Office of State Printer and moved to the state capitol , Sa
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
espectively . Hughes , trained as a printer , became co-owner of the < a href= 
no matches
Displaying 4 of 4 matches:
er achieving success as a newspaper printer in Baltimore , Matthew Brown ( 1774
 1774-1831 ) , Philadelphia-trained printer , hoped to launch a Federalist

Displaying 1 of 1 matches:
s both a grammar school teacher and printer . In March 1834 , he disseminated t
Displaying 1 of 1 matches:
s both a grammar school teacher and printer . In March 1834 , he disseminated t
Displaying 1 of 1 matches:
s both a grammar school teacher and printer . In March 1834 , he disseminated t
Displaying 1 of 1 matches:
s both a grammar school teacher and printer . In March 1834 , he disseminated t
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
t of his life . He apprenticed as a printer , served in the Civil War , and the
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 2 of 2 matches:
emained publisher until 1945 and as printer of the < em > Fair P

no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 2 of 2 matches:
arly 18 years , his son rising from printer & # 8217 ; s devil to manager and e
Jr. , began his newspaper career as printer & # 8217 ; s devil . Upon Kinner Sr
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
< /cite > . In 1906 , a disgruntled printer left Live Oak by railroad on the ni
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 2 of 2 matches:
. Shanks . Shanks w

no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
s considered an able journalist and printer , Rowlenson changed the name of the
no matches
Displaying 2 of 2 matches:
e W. Benedict , the Canadian-reared printer whom they lured from his Tecumseh ,
1 ; George Benedict remained as its printer , and he went on to a Minnesota jou
no matches
no matches
no matches
no matches
no matches
Displaying 2 of 2 matches:
st of his life . He was a prominent printer and publisher in the state of Washi
 , he was appointed to be the state printer by Governor Monrad C. Wallgren , wh
no matches
no matches
no matches
no matches
Displaying 1 of 1 matche

0       None
1       None
2       None
3       None
4       None
        ... 
3675    None
3676    None
3677    None
3678    None
3679    None
Name: essay, Length: 3680, dtype: object

-------

Another aspect that thinks more deeply about extracting information from title essay content utilizes `spacy's` language models to isolate named identities such as people, organization, and locations within the unstructured content of the essays.

The code below extracts people and organizations from the title essays. The commented text also includes starter code to extract noun phrases or locations.

NOTE: This code takes a few minutes to run.

In [28]:
df['essay']= df['essay'].astype(str)
ppl_df = pd.DataFrame()


for row in range(len(df)):
    people = {}
    lccn = df.loc[row, "raw_lccn"]
    doc = nlp(df.loc[row, "essay"])
#     for np in doc.noun_chunks:
#         print(np.text)
    person = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
    orgs = [ent.text for ent in doc.ents if ent.label_ == 'ORG']
    lccn = str(lccn).strip()
    people[lccn] = [person, orgs]
    t = pd.DataFrame.from_dict(people, orient='index', columns=["people","organization"])
    ppl_df = pd.concat([ppl_df, t], ignore_index = True)

#     test['person'] = docs.ents.apply(ent.text if ent.label == 'PERSON')
#     orgs = [ent.text for ent in doc.ents if ent.label_ == 'ORG']
#     LOC = [ent.text for ent in doc.ents if ent.label_ == 'LOC']
#     person = pd.DataFrame(person)
#     print(person)
#     test['person'] = person.loc[row, "essay"]


In [30]:
ppl_df.tail(50)

,people,organization
3630,"[Whigs, Rapids, Centralia, John N. Brundage, James E. Ingraham, Ingraham, Philleo, Paul, Albert L. Fontaine, Paul, Albert]","[the Republican Party, Army, the Republican Party, Nekoosa, Christian Temperance Union, &#]"
3631,"[Broadford, Marshall]","[Hailey, Hailey, Bellevue, Bullion, mills.&#8221, Hailey, the anti-Chinese League]"
3632,"[William Edward Woodruff, Cyrus McCrory, Wade McCrory, Cotton, Wade McCrory, Great, Gustave W. Kramer, Kramer, Walter Wilson, Raney, Raney, Raney, Gustave Kramer, Ernest Carl Kramer, Ernest Kramer, Charles M. James, James, Raney, Raney, Charles James, James]","[href=""https://chroniclingamerica.loc.gov, the Bank of McCrory, href=""https://chroniclingamerica.loc.gov, the Arkansas Supreme Court, the News Publishing Company, Kramer, Raney, McCrory]"
3633,"[Martin Van Buren, Van Buren, Charles G. Eastman, Eastman, Edgar A. Kimball, Kimball, Edward M. Brown, Brown, Montpelier, Charles, William D. McMaster, Edward C. Dana, Eastman, Kimball, Brown, McMaster, McMaster, Edward Dana]","[Democratic Party, Kimball, the opposition&#8217;s &#8220;black, the Elm Tree Press]"
3634,"[Woodville, Jefferson Davis, William A. A. Chisholm, Wilkinson Advertiser</em>, William H. Nobles, John South Lewis, Andrew Lewis, John S., Theodore]","[the Old Natchez Cession, Rosemont Plantation.</p>, Chisholm, Weekly Advertiser, the Democratic Party, Confederate, the Nashville Convention, Journal of Wilkinson]"
3635,"[Woodville, Jefferson Davis, William A. A. Chisholm, Wilkinson Advertiser</em>, William H. Nobles, John South Lewis, Andrew Lewis, John S., Theodore]","[the Old Natchez Cession, Rosemont Plantation.</p>, Chisholm, Weekly Advertiser, the Democratic Party, Confederate, the Nashville Convention, Journal of Wilkinson]"
3636,"[Woodville, Jefferson Davis, William A. A. Chisholm, Wilkinson Advertiser</em>, William H. Nobles, John South Lewis, Andrew Lewis, John S., Theodore]","[the Old Natchez Cession, Rosemont Plantation.</p>, Chisholm, Weekly Advertiser, the Democratic Party, Confederate, the Nashville Convention, Journal of Wilkinson]"
3637,"[Woodville, Jefferson Davis, William A. A. Chisholm, Wilkinson Advertiser</em>, William H. Nobles, John South Lewis, Andrew Lewis, John S., Theodore]","[the Old Natchez Cession, Rosemont Plantation.</p>, Chisholm, Weekly Advertiser, the Democratic Party, Confederate, the Nashville Convention, Journal of Wilkinson]"
3638,"[Samuel M. Crockett, Samuel M. Crockett, Bay craft, Edd Watson, H.L. Mencken, Matthew Williams, Mencken, George Armwood]","[href=""https://chroniclingamerica.loc.gov, href=""https://chroniclingamerica.loc.gov, href=""https://chroniclingamerica.loc.gov, href=""https://chroniclingamerica.loc.gov, W.B. Vane, href=""https://chroniclingamerica.loc.gov, href=""https://chroniclingamerica.loc.gov, the Maryland, href=""https://chroniclingamerica.loc.gov, Newtown, city.&#8221, Eastern Shore, href=""https://chroniclingamerica.loc.gov]"
3639,"[Andrew Cameron, Cameron]","[the Chicago Typographical Union, href=""https://chroniclingamerica.loc.gov, href=""https://chroniclingamerica.loc.gov, Evergreen, the Typographical Union, Cameron, the National Labor Union, NLU, NLU, cooperatives, nation&#8217;s, NLU, the Industrial Congress of the United States]"


In [33]:
df_cd = pd.concat([df, ppl_df], axis=1)

In [34]:
df_cd

created_published  \
0                                ['Abbeville, S.C. : Charles H. Allen, 1847-1869.']   
1                                                 ['Abbeville, S.C. : Hugh Wilson']   
2                                           ['Abbeville, S.C. : Bonham and Perrin']   
3                          ['Abbeville, S.C. : W.A. Lee & Hugh Wilson, 1869-1924.']   
4                   ['Abbeville, S.C. : W.A. Lee and Hugh Wilson, Jr., 1860-1869.']   
...                                                                             ...   
3675                                     ['Allegheny, Pa. : Narod. Hrv. Zajednica']   
3676                       ['Chicago, Ill. : Polish National Alliance, 1900-1913.']   
3677                       ['Chicago, Ill. : Polish National Alliance, 1902-1913.']   
3678  ['Chicago, Illinois : Polish National Alliance of the U. S. of N. A., 1914-']   
3679                                 ['Milwaukee, Wis. : Polish National Alliance']   

      date dates_of_publication  \
0     1847            1847-1869   
1     1865            1865-1865   
2     1884            1884-1887   
3     1869            1869-1924   
4     1860            1860-1869   
...    ...                  ...   
3675  1894         1894-current   
3676  1900            1900-1913   
3677  1902            1902-1913   
3678  1914         1914-current   
3679  1881            1881-1901   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           description  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         ['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, no. 47 (Sept. 29, 1869). Publication suspended, <1861-1865>. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Abbeville press (DLC)sn 85042527 (OCoLC)12622573 Abbeville press and banner (DLC)sn 84026853 (OCoLC)10708843']   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [26]:
df_cd = pd.merge(df, ppl_df, how='inner', left_on = 'raw_lccn', right_index=True)

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [17]:
df_cd.head()

created_published  \
91         ['Westerville, Ohio : American Issue Pub. Co., 1912-[19--?]']   
324  ['Brookings, Dakota [S.D.] : Geo. N. Breed and Chas D. A. Wright,']   
325                               ['Brookings, S.D. : Dutcher & Breed,']   
327  ['Brookings, Brookings County, S.D. : Dutcher, Breed & Storgaard,']   
684                             ['Corvallis, Or. : B.F. Irvine, -1909.']   

     date dates_of_publication  \
91   1912            1912-19??   
324  1882            1882-1890   
325  1890            1890-1894   
327  1898            1898-1903   
684  1888            1888-1909   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  description  \
91                                                                                                                  ['Monthly (except Aug.), Oct. 1925-<May 1932> Vol. 7, no. 7 (Feb. 16, 1912)- Preservation microfilmed by the University of Illinois at Urbana-Champaign Library as part of the United States Newspaper Program; the years 1912-1932 (on 5 microfilm reels) are available for purchase from OCLC Preservation Service Centers. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Published for: the Anti-Saloon League of Illinois. Latest issue consulted: Vol. 27, no. 5 (May 1932).']   
324                                                                                                                                                                                                                                                                                                                                                                                                                                     ['Weekly Vol. 1, no. 1 (Mar. 30, 1882)-v. 9, no. 41 (Dec. 26, 1890). Available on microfilm from: State Archives, South Dakota State Historical Society. Brookings register (Brookings, S.D. : 1890) Jan. 6, 1891 (DLC)sn 200163544']   
325                                                                                                                                                                                                                                                                                                                                                                                                                                                      ['Weekly Vol. 1, no. 1 (June 13, 1890)-v. 14, no. 20 (Oct. 19, 1894). Available on microfilm from: State Archives, South Dakota State Historical Society. Semi-weekly register (Brookings, S.D.) (DLC)sn 200163545']   
327                                                                                                                                                                                                                                                                                                                                                                                                                                                ['Weekly Vol. 18, no. 28 (Dec. 15, 1898)-v. 23, no. 14 (Sept. 3, 1903). Available on microfilm from: State Archives, South Dakota State Historical Society. Brookings register (Brookings, S.D. : 1903) (DLC)sn 85042792']   
684  ['Semiweekly, <May 28, 1894>-June 25, 1909 Began in 1888. Cf. Turnbull, G., History of Oregon newspaper